
# MCM 项目：多模态入侵物种检测系统
**美国大学生数学建模竞赛 (MCM) 项目文档**

本笔记本记录了我们的亚洲大黄蜂检测系统的核心逻辑、数学模型和验证结果。
该系统集成了 **文本分析 (NLP)**、**图像特征提取** 和 **时空分析**，以对用户提交的目击报告进行分类。

---



## 第一部分：文本评分模型 (NLP)
**参考文件**: `text_scoring_model.py`

### 1.1 模型逻辑
为了确保 **可解释性**，我们采用了 **基于贝叶斯的关键词加权** 方法，而不是黑盒深度学习模型。
该模型根据单词在 `Positive ID`（确认）与 `Negative ID`（排除）报告中出现的对数几率比 (Log-Odds Ratio) 为单词分配权重。

### 1.2 数学公式
对于词汇表中的每个单词 $w$，其权重 $W_w$ 计算如下：

$$
W_w = Base_w + \alpha \cdot \log\left(\frac{P(w|Positive) + \epsilon}{P(w|Negative) + \epsilon}\right) + \beta \cdot I(w \in Theory)
$$

其中：
- $Base_w$: 基于专家规则的初始启发式得分（例如，"confirmed" = +15）。
- $\log(\dots)$: 对数几率比 (LOD)，捕捉统计相关性。
- $I(w \in Theory)$: 指示函数，如果单词在我们预定义的生物学关键词列表中，则为 1。
- $\alpha, \beta$: 超参数（分别设置为 5.0 和 3.0）。

一份报告的最终文本得分 $S_{text}$ 是单词权重之和的 Sigmoid 激活函数值：

$$
S_{text} = \frac{1}{1 + e^{-k( \sum W_w - \theta )}}
$$


In [ ]:

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# 设置绘图风格以支持中文显示（如果环境支持）
plt.rcParams['font.sans-serif'] = ['SimHei', 'Arial Unicode MS'] 
plt.rcParams['axes.unicode_minus'] = False

# 加载数据
try:
    df_text = pd.read_csv('Cleaned_Data_Scored.csv')
    print("已加载 Cleaned_Data_Scored.csv")
except FileNotFoundError:
    print("未找到数据文件。请先运行文本模型脚本。")
    df_text = pd.DataFrame({'Lab Status': [], 'Text_Model_Score': []})

# 可视化：得分分布
if not df_text.empty:
    plt.figure(figsize=(10, 6))
    sns.histplot(data=df_text, x='Text_Model_Score', hue='Lab Status', 
                 hue_order=['Positive ID', 'Negative ID', 'Unverified'],
                 kde=True, element="step", common_norm=False)
    plt.title('文本模型得分分布 (按实验室状态分类)')
    plt.xlabel('模型概率得分')
    plt.ylabel('数量')
    plt.show()



## 第二部分：图像特征提取与匹配 **[新增]**
**参考文件**: `image_feature_extraction.py`

### 2.1 方法论
为了在不训练大型 CNN 的情况下验证图像，我们使用了可解释的 **手工特征 (Handcrafted Features)** 结合 **重心匹配模型 (Centroid Matching Model)**。
这种方法对小数据集具有很强的鲁棒性（我们的 Positive 样本非常有限）。

### 2.2 特征维度
1.  **纹理 (GLCM)**: 灰度共生矩阵，用于检测黄蜂腹部特定的条纹图案。
    *   **对比度 (Contrast)**: $\sum_{i,j} |i-j|^2 p(i,j)$ (反映条纹边缘的清晰度)
    *   **熵 (Entropy)**: $-\sum_{i,j} p(i,j) \log(p(i,j))$ (反映图案的复杂度)
2.  **颜色 (HSV)**: Hue（色调）、Saturation（饱和度）、Value（明度）空间的直方图统计量。
    *   亚洲大黄蜂具有独特的橙/黄色头部和深色胸部。

### 2.3 重心匹配公式
我们根据已验证的阳性图像计算“标准黄蜂”的特征重心向量 $\vec{C}$。
对于一个新的图像向量 $\vec{x}$，其相似度得分 $S_{img}$ 计算如下：

$$
D = || \vec{x} - \vec{C} ||_2 \quad (\text{欧氏距离})
$$

$$
S_{img} = 1 - \frac{D}{D_{max}}
$$

这将得分归一化到 $[0, 1]$ 区间，其中 1 表示与重心完美匹配。


In [ ]:

# 加载图像特征
try:
    df_img = pd.read_csv('Image_Features_Scored.csv')
    print("已加载 Image_Features_Scored.csv")
except FileNotFoundError:
    print("未找到图像特征文件。")
    df_img = pd.DataFrame()

if not df_img.empty:
    # 1. 特征相关性热力图
    plt.figure(figsize=(10, 8))
    cols = ['GLCM_Contrast', 'GLCM_Energy', 'GLCM_Correlation', 'GLCM_Entropy', 
            'HSV_H_Mean', 'HSV_S_Mean', 'Image_Score']
    sns.heatmap(df_img[cols].corr(), annot=True, cmap='coolwarm', fmt=".2f")
    plt.title('图像特征相关性矩阵')
    plt.show()

    # 2. 得分箱线图
    plt.figure(figsize=(8, 6))
    sns.boxplot(x='Lab Status', y='Image_Score', data=df_img)
    plt.title('图像相似度得分 (按实验室状态分类)')
    plt.show()



## 第三部分：综合决策与鲁棒性
**参考文件**: `决策+扩散+优先级invasion_model.py` (入侵模型)

### 3.1 融合逻辑
最终的验证决策结合了来自三个来源的证据：**位置 (空间)**、**文本** 和 **图像**。
这种多模态方法提高了鲁棒性：即使某一模态失效（例如照片模糊），其他模态也能进行补偿。

$$
Score_{final} = R_{user} \times (w_L \cdot S_{loc} + w_T \cdot S_{text} + w_V \cdot S_{img})
$$

其中：
- $R_{user}$: 报告者的贝叶斯信誉评分（初始值为 0.5，随历史记录更新）。
- $S_{loc}$: 若在已知感染半径内为 1.0，否则为 0.4。
- 权重: $w_L=0.3, w_T=0.3, w_V=0.4$ (视觉证据权重最高)。

### 3.2 鲁棒性检验
模型的鲁棒性体现在：
1.  **缺失数据处理**: 如果缺少图像，$S_{img}$ 默认为中性低值，主要依赖文本和位置。
2.  **用户信誉机制**: 重复的误报会降低用户的 $R_{user}$，从而随着时间的推移过滤掉噪声。


In [ ]:

# 散点图：文本得分 vs 图像得分
# 理想情况下，Positive ID 样本应聚集在右上角（高文本得分，高图像得分）

if not df_img.empty and not df_text.empty:
    plt.figure(figsize=(10, 8))
    
    # 注意：这里直接使用 df_img 进行绘图，假设其中已包含 Lab Status 信息
    # 实际应用中可能需要先 merge df_text 和 df_img
    
    sns.scatterplot(data=df_img, x='Centroid_Distance', y='Image_Score', hue='Lab Status', style='Lab Status', s=100)
    plt.title('图像得分 vs 重心距离 (分离度检查)')
    plt.grid(True, alpha=0.3)
    plt.show()
